In [ ]:
import pandas as pd
import dash
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
import plotly.express as px
import json
import geojson
import math


df_dejavnosti = pd.read_csv("data/podatki_po_dejavnostih.csv", delimiter=",", encoding="windows-1250")
df_odjemalci = pd.read_csv("data/podatki_vrsta_odjemalca.csv", delimiter=",", encoding="windows-1250")

In [ ]:
df_odjemalci_backup = df_odjemalci.copy()

#Tukaj vzamemo samo podatke za VSE SKUPAJ porabo (brez gospodinjstva in poslovnih objektov)
df = df_odjemalci_backup[::+3]
#Tukaj izbrišem podatke za SLOVENIJO in Neznano vrstice
df = df[1:-1]

#Priprava podatkov za uporabo
df = df.drop("VRSTA ODJEMALCA", axis=1)
obcine = {df["OBČINE"][i]:i for i in df.index}

vse_2020 = dict(df[df.columns[1:13]].sum(axis=1))

dict_porab = dict([(k,vse_2020[k]) for k in vse_2020])

#V koncem dictu so shranjene OBČINE (ključ) z vrednostmi, ki so LISTI, 
# ki prikazujejo porabo za vsako občino za vsako leto posebaj
koncni_dict = dict()
for k,v in obcine.items():
    koncni_dict[k] = dict_porab[v]


In [ ]:
def izrisi_odjemalci_Slovenija(odjemalci, leto):
    df_odjemalci = odjemalci.copy()
    df=df_odjemalci[::+3]
    df = df[1:-1]
    df = df.drop("VRSTA ODJEMALCA", axis=1)
    obcine = {df["OBČINE"][i]:i for i in df.index}

    dict_porab = dict()
    if leto == 2020:
        vse_2020 = dict(df[df.columns[1:13]].sum(axis=1))
        dict_porab = dict([(k,[vse_2020[k]]) for k in vse_2020])
    elif leto == 2021:
        vse_2021 = dict(df[df.columns[13:25]].sum(axis=1))
        dict_porab = dict([(k,[vse_2021[k]]) for k in vse_2021])

    koncni_dict = dict()
    for k,v in obcine.items():
        koncni_dict[k] = dict_porab[v]
    
    return pd.DataFrame.from_dict(koncni_dict).transpose()

In [ ]:
def izrisi_odjemalci_gospodinjstva(odjemalci, leto):
    df_odjemalci = odjemalci.copy()
    df=df_odjemalci[1::+3]
    df = df[1:-1]
    df = df.drop("VRSTA ODJEMALCA", axis=1)
    obcine = {df["OBČINE"][i]:i for i in df.index}

    dict_porab = dict()
    if leto == 2020:
        vse_2020 = dict(df[df.columns[1:13]].sum(axis=1))
        dict_porab = dict([(k,[vse_2020[k]]) for k in vse_2020])
    elif leto == 2021:
        vse_2021 = dict(df[df.columns[13:25]].sum(axis=1))
        dict_porab = dict([(k,[vse_2021[k]]) for k in vse_2021])

    koncni_dict = dict()
    for k,v in obcine.items():
        koncni_dict[k] = dict_porab[v]

 
    return pd.DataFrame.from_dict(koncni_dict).transpose()


In [ ]:
def izrisi_odjemalci_poslovni_objekti(odjemalci, leto):
    df_odjemalci = odjemalci.copy()
    df=df_odjemalci[2::+3]
    df = df[1:-1]
    df = df.drop("VRSTA ODJEMALCA", axis=1)
    obcine = {df["OBČINE"][i]:i for i in df.index}

    dict_porab = dict()
    if leto == 2020:
        vse_2020 = dict(df[df.columns[1:13]].sum(axis=1))
        dict_porab = dict([(k,[vse_2020[k]]) for k in vse_2020])
    elif leto == 2021:
        vse_2021 = dict(df[df.columns[13:25]].sum(axis=1))
        dict_porab = dict([(k,[vse_2021[k]]) for k in vse_2021])

    koncni_dict = dict()
    for k,v in obcine.items():
        koncni_dict[k] = dict_porab[v]

    return pd.DataFrame.from_dict(koncni_dict).transpose()

In [ ]:
#Izris podatkov PRIMERI (2020 in 2021)
df = izrisi_odjemalci_Slovenija(df_odjemalci, 2020)
#izrisi_odjemalci_gospodinjstva(df_odjemalci, 2020)
#izrisi_odjemalci_poslovni_objekti(df_odjemalci, 2020)

In [ ]:
map_df1 = gpd.read_file('slovenija_map/obcine/obcine.shp')
map_df1 = map_df1.to_crs("WGS84")
map_df1 = map_df1.rename({'NAZIV': 'District'}, axis = 'columns')
map_df1 = map_df1.drop(columns = ['EID_OBCINA',  'SIFRA', 'NAZIV_DJ', 'OZNAKA_MES', 'DATUM_SYS'])
map_df1 = map_df1.sort_values('District')

list(map_df1["District"])

In [ ]:
df = df.sort_index()
list(df.index)

In [ ]:
map_df = gpd.read_file('slovenija_map/obcine/obcine.shp')
map_df = map_df.to_crs("WGS84")
map_df = map_df.rename({'NAZIV': 'District'}, axis = 'columns')
map_df = map_df.drop(columns = ['EID_OBCINA',  'SIFRA', 'NAZIV_DJ', 'OZNAKA_MES', 'DATUM_SYS'])
map_df = map_df.sort_values('District')


map_df.loc[map_df["District"] == "Ljubljana"]

In [ ]:
list(df[0].values)

In [ ]:
def uredi_data_prebivalci(prebivalci_data, mapdf):
    df = prebivalci_data.copy()
    df = df.rename({'OBČINE': 'District'}, axis = 'columns')
    df = df.drop(df.index[0])

    df.index = mapdf.index
    df["District"] = mapdf["District"]

    df["2020_skupaj"] = ((df["2020H1 Starost - SKUPAJ"] + df["2020H2 Starost - SKUPAJ"]) / 2).astype(int)
    df["2021_skupaj"] = ((df["2021H1 Starost - SKUPAJ"] + df["2021H2 Starost - SKUPAJ"]) / 2).astype(int)
    df = df.drop(columns = ["2021H1 Starost - SKUPAJ",  '2021H2 Starost - SKUPAJ', '2020H1 Starost - SKUPAJ', '2020H2 Starost - SKUPAJ'])

    return df

In [ ]:
df_prebivalci = pd.read_csv("data/podatki_starost_prebivalcev.csv", delimiter=",", encoding="windows-1250")


def izracun_normalizacije_Slovenija(mapdf, leto):
    urejeni = uredi_data_prebivalci(df_prebivalci, map_df)
    df = izrisi_odjemalci_Slovenija(df_odjemalci, leto)
    df.insert(0,'District', df.index)
    df.index = mapdf.index
    if leto == 2020:
        df["Poraba"] = round(df[0] / urejeni["2020_skupaj"],2)
    elif leto == 2021:
        df["Poraba"] = round(df[0] / urejeni["2021_skupaj"],2)

    df = df.drop(columns={0})
    return df

def izracun_normalizacije_gospodinjstva(mapdf, leto):
    urejeni = uredi_data_prebivalci(df_prebivalci, map_df)
    df = izrisi_odjemalci_gospodinjstva(df_odjemalci, leto)
    df.insert(0,'District', df.index)
    df.index = mapdf.index
    if leto == 2020:
        df["Poraba"] = round(df[0] / urejeni["2020_skupaj"],2)
    elif leto == 2021:
        df["Poraba"] = round(df[0] / urejeni["2021_skupaj"],2)

    df = df.drop(columns={0})
    return df

def izracun_normalizacije_poslovni_objekti(mapdf, leto):
    urejeni = uredi_data_prebivalci(df_prebivalci, map_df)
    df = izrisi_odjemalci_poslovni_objekti(df_odjemalci, leto)
    df.insert(0,'District', df.index)
    df.index = mapdf.index
    if leto == 2020:
        df["Poraba"] = round(df[0] / urejeni["2020_skupaj"],2)
    elif leto == 2021:
        df["Poraba"] = round(df[0] / urejeni["2021_skupaj"],2)

    df = df.drop(columns={0})
    return df

#izracun_normalizacije_Slovenija(map_df, 2020)
#izracun_normalizacije_gospodinjstva(map_df, 2020)
#izracun_normalizacije_poslovni_objekti(map_df, 2021)


In [ ]:
map_df